In [ ]:
import subprocess
import sys
from siphon.simplewebservice.ndbc import NDBC
import time
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
import os,json
from pathlib import Path
import meteomatics.api as api
import matplotlib.pyplot as plt
import pytz, folium, gspread
from datetime import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from requests.exceptions import HTTPError  # Importer HTTPError
import sys
from urllib.error import HTTPError
from sklearn.preprocessing import StandardScaler


def install_and_import(*packages):
    for package in packages:
        try:
            # Tenter d'importer chaque bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' est déjà installé et importé.")
        except ImportError:
            # Si l'import échoue, installer la bibliothèque
            print(f"Le package '{package}' n'est pas installé. Installation en cours...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            # Ensuite, essayer à nouveau d'importer la bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' a été installé et importé.")

%pip install pyowm
# Utilisation de la fonction pour installer et importer plusieurs packages
install_and_import("dask", "pyowm", "ndbc_api", "plotly")
import plotly.express as px

In [ ]:
# Définir les scopes requis pour accéder à Google Sheets et Google Drive
scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = "google_sheets_api.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# ID de la feuille Google Sheets
sheet_id = "1rXyD4WW5DcrO2iCywfwLbqps3Z-RumxQH9hJofhVM24/"

# Ouvrir la feuille Google Sheets en utilisant son ID
spreadsheet = client.open_by_key(sheet_id)

worksheets = spreadsheet.worksheets()

drive_service = build('drive', 'v3', credentials=creds)

Get Data from APIs

Marine API

In [ ]:
from ndbc_api import NdbcApi

api = NdbcApi()

# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()

stations_df.head()

Get Nearest

In [69]:
latitude = 11.2404
longitude = -74.211

nearest = api.nearest_station(lat=latitude, lon=longitude)
nearest
for index, row in stations_df.iterrows():
    if row['Station'] == '42058':
        lat_buoy_near_SM = row['Lat']
        lon_buoy_near_SM = row['Lon']
        print(lat_buoy_near_SM)
        print(lon_buoy_near_SM)
df_marine = NDBC.realtime_observations(nearest)


14.512
-75.153


In [73]:
# Créer la carte centrée sur les coordonnées
m = folium.Map(location=[latitude, longitude], zoom_start=7)

# Ajouter un marqueur sur Buoy_51101
marker1 = folium.Marker([latitude, longitude]).add_to(m)
marker2 = folium.Marker([lat_buoy_near_SM, lon_buoy_near_SM], popup="Buoy closest to SM").add_to(m)

m

In [ ]:
caribbean_df = stations_df[(stations_df['Lat'] >= 9) & 
                           (stations_df['Lat'] <= 25) & 
                           (stations_df['Lon'] >= -85) & 
                           (stations_df['Lon'] <= -60)]
caribbean_df.shape[0]

In [ ]:
buoys_ids = []

for index, row in caribbean_df.iterrows():
    buoys_ids.append(row['Station'])  
print(len(buoys_ids))    

In [ ]:
# Liste pour stocker les dataframes
all_dataframes = []

# Compteurs pour les bouées réussies et échouées
successful_buoy = 0
failed_buoy = 0

# Parcours de chaque bouée
for buoy_id in buoys_ids:
    try:
        # Récupère les observations en temps réel pour chaque bouée
        df_buoy = NDBC.realtime_observations(buoy_id)

        # Ajoute le dataframe à la liste
        all_dataframes.append(df_buoy)
        successful_buoy += 1

        # Efface et met à jour la ligne des bouées réussies
        print(f"\r✅ Bouées réussies : {successful_buoy}", end='', flush=True)

    except Exception:
        # Incrémente le compteur des échouées pour toutes les erreurs
        failed_buoy += 1

        # Efface et met à jour la ligne des bouées échouées
        print(f"\r❌ Bouées échouées : {failed_buoy}", end='', flush=True)

# Affichage final propre avec un saut de ligne
print("\n")

# Concaténation des dataframes si disponibles
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"✔️ Concaténation terminée. Nombre de lignes : {final_df.shape[0]} \n Nombre de colonnes : {final_df.shape[1]}")
    print(f"✅ Bouées réussies : {successful_buoy}", end='', flush=True)
    print(f"❌ Bouées échouées : {failed_buoy}", end='', flush=True)

else:
    print("⚠️ Aucune donnée récupérée.")


In [ ]:
missing_percent = round((final_df.isnull().sum() / len(final_df)) * 100, 2)
missing_percent_str = missing_percent.astype(str) + '%'
print(missing_percent_str.sort_values(ascending=False))


In [ ]:
def count_null_values(df) :
    print(f"For {df.shape[0]} rows : \n")
    for col in df.columns:
        col_null_values_count = df[col].isnull().sum()
        print(f" '{col}' has {col_null_values_count} null values \n")
count_null_values(final_df)

In [ ]:
def handle_null_values(df):
    # Calcul des pourcentages de valeurs manquantes
    missing_percent = round((df.isnull().sum() / len(df)) * 100, 2)
    missing_percent_str = missing_percent.astype(str) + '%'
    
    # Afficher les colonnes et leurs pourcentages de valeurs manquantes
    print("Pourcentages de valeurs manquantes par colonne :")
    print(missing_percent_str.sort_values(ascending=False))
    
    # Gestion des valeurs manquantes
    for column in df.columns:
        null_percentage = missing_percent[column]
        
        if null_percentage == 100:
            # Si 100% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} (100% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 50:
            # Si plus de 50% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} ({null_percentage}% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 0:
            # Si moins de 50% des valeurs sont manquantes, on impute avec la médiane
            print(f"Impute la colonne : {column} avec la médiane ({null_percentage}% de valeurs manquantes)")
            median_value = df[column].median()
            df[column] = df[column].fillna(median_value)
    
    # Retourner le DataFrame modifié
    return df

# Exemple d'appel de la fonction
final_df = handle_null_values(final_df)

print(final_df.shape)
final_df.head()

In [ ]:

# Convertir la colonne 'time' en datetime et la garder comme une colonne
df_marine['time'] = pd.to_datetime(df_marine['time'])

# Configurer 'time' comme index, puis resampler
df_marine.set_index('time', inplace=True)

# Resampler à l'heure et prendre la moyenne
df_marine = df_marine.resample('h').mean()

# Optionnel : Si tu veux garder 'time' comme une colonne, on peut le réinitialiser
df_marine.reset_index(inplace=True)

# Afficher le résultat
df_marine.head()

In [ ]:
# Arrondir toutes les colonnes numériques (int ou float) à 2 chiffres après la virgule
df_marine = df_marine.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

# Vérifier le résultat
df_marine.head()

In [ ]:
lat = 24.359
lon = -162.081

Meteomatics API

In [ ]:
def read_credentials():
    with open('credentials.json', 'r') as file:
        credentials = json.load(file)
    
    meteo_username = credentials.get('username')
    meteo_password = credentials.get('password')
    
    return meteo_username, meteo_password


In [ ]:
# Définir les paramètres
parameters = [
        'wind_speed_10m:ms',	
        'wind_dir_10m:d',
        'wind_gusts_10m_1h:ms',
        't_2m:C',
        't_max_2m_24h:C',
        't_min_2m_24h:C',
        'msl_pressure:hPa',
        'precip_1h:mm'
    ]

# Définir les coordonnées
coordinates = [(lat, lon)]

# Définir le modèle
model = 'mix'
# Début URL de l'API
DEFAULT_API_BASE_URL = "https://api.meteomatics.com"
enddate = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
startdate = enddate - timedelta(days=1)
interval = timedelta(hours=1)

# Fonction de requête API Météo
def meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval):

    # Appel de la fonction read_credentials pour obtenir les informations d'identification
    meteo_username, meteo_password = read_credentials()
    
    # Récupérer les données
    df_meteo = api.query_time_series(coordinates, startdate, enddate, interval, parameters, meteo_username, meteo_password, model=model)
    df_meteo.reset_index(inplace=True)
    return df_meteo

df_meteo=meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval)
df_meteo = df_meteo.rename(columns={'validdate': 'time'})
df_meteo.head()

In [ ]:
df_marine.head()

Check first rows

In [ ]:
def convert_to_datetime(value):
    try:
        # Spécifier dayfirst=True pour éviter le warning avec le format %d/%m/%Y
        return pd.to_datetime(value, errors='coerce', dayfirst=True).strftime('%d/%m/%Y-%H:%M') if pd.notna(value) else None
    except Exception as e:
        return None  # Retourner None si la conversion échoue


# Appliquer la fonction sur la colonne 'time' avec apply
df_marine['time'] = df_marine['time'].apply(convert_to_datetime)
df_meteo['time'] = df_meteo['time'].apply(convert_to_datetime)

Rename Columns + Delete Useless Ones

In [ ]:
try:
    # Renommage des colonnes pour df_marine
    rename_cols = {
        'wind_direction': 'Wind Dir (°)', 'wind_speed': 'Wind Speed (ms)',
        'wind_gust': 'Wind Gusts 1h (ms)', 'wave_height': 'Wave Height (m)',
        'dominant_wave_period': 'Dominant Wave Period (s)', 'average_wave_period': 'Avg Wave Period (s)',
        'dominant_wave_direction': 'Dominant Wave Direction', 'air_temperature': 'Air Temperature',
        'water_temperature': 'Water Temperature', 'time': 'Datetime', 'pressure': 'Pressure'
    }
    
    # Renommer avec vérif
    for old_col, new_col in rename_cols.items():
        if old_col in df_marine.columns:
            df_marine.rename(columns={old_col: new_col}, inplace=True)
        else:
            print(f"Colonne non trouvée pour renommage : {old_col}")

    # Supprimer les colonnes inutiles
    for col in ['dewpoint', 'visibility', '3hr_pressure_tendency', 'water_level_above_mean', 'lat', 'lon']:
        if col in df_marine.columns:
            df_marine.drop(columns=col, inplace=True)
        else:
            print(f"Colonne non trouvée pour suppression : {col}")

    # Pour df_meteo
    for col in ['lat', 'lon']:
        if col in df_meteo.columns:
            df_meteo.drop(columns=col, inplace=True)
        else:
            print(f"Colonne non trouvée dans df_meteo : {col}")

    # Renommer 'time' en 'Datetime' dans df_meteo
    if 'time' in df_meteo.columns:
        df_meteo.rename(columns={'time': 'Datetime'}, inplace=True)
    else:
        print("Colonne 'time' non trouvée dans df_meteo pour renommage.")

except KeyError as e:
    print(f"Erreur de clé : {e}")


In [ ]:
df_meteo.head()

In [ ]:
print(f"Marine Data Rows Amount : {df_marine.shape[0]}")
marine_null_counts = df_marine.isnull().sum()

print(f"Marine Data Null Values Amount: \n{marine_null_counts}")


In [ ]:
def handle_null_values(df):
    # Itérer sur les colonnes du DataFrame
    for column in df.columns:
        # Si la colonne est numérique, on remplace les valeurs nulles par la médiane
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column] = df[column].fillna(df[column].median())  # Remplacer sans inplace=True
        # Sinon, on supprime les lignes où il y a des valeurs nulles dans cette colonne
        else:
            df.dropna(subset=[column], inplace=True)
    
    return df  # Retourner le DataFrame après traitement

# Exemple d'utilisation
df_marine = handle_null_values(df_marine)
df_meteo = handle_null_values(df_meteo)

# Vérifier le résultat
df_marine.isnull().sum() # Afficher les valeurs nulles restantes
df_meteo.isnull().sum() # Afficher les valeurs nulles restantes

In [ ]:
df_meteo.head()

In [ ]:
print(f"Number of Rows in marine DataFrame : {df_marine.shape[0]}")
print(f"Number of Rows in meteo DataFrame : {df_meteo.shape[0]}")

Merging Dataframes

In [ ]:
intersection_df = pd.merge(df_marine, df_meteo)
intersection_df.head()

In [ ]:
# Effectuer le merge entre df_marine et df_meteo sur la colonne 'time'
df_result = pd.merge(df_marine, df_meteo, on='Datetime', how='right')  # 'inner' pour ne garder que les lignes avec des 'time' correspondants

# Afficher le nombre de lignes de la DataFrame résultante
num_rows = df_result.shape[0]
print(f"Nombre de lignes dans la DataFrame résultante : {num_rows}")


In [ ]:
columns_to_drop = ['lat', 'lon']

def drop_columns(df, columns_to_drop):
    for col in columns_to_drop:
        if col in df.columns:
            df = df.drop(columns=col)
        else:
            print(f"'{col}' is not found in the DataFrame")  # Affiche au lieu de retourner une chaîne
    return df  # Toujours retourner le DataFrame

df = drop_columns(df_result, columns_to_drop)

df.head()


In [ ]:
# Convertir 'Datetime' en format datetime
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d/%m/%Y-%H:%M')

# Définir une fonction pour classifier les périodes de la journée
def get_day_period(hour):
    if 5 <= hour < 12:
        return 'Matin'
    elif 12 <= hour < 17:
        return 'Midi'
    elif 17 <= hour < 21:
        return 'Soir'
    else:
        return 'Nuit'

# Appliquer la fonction et créer la nouvelle colonne
df['Day Period'] = df['Datetime'].dt.hour.apply(get_day_period)

df = drop_columns(df, ['Datetime'])
# Voir le résultat
df.head()

In [ ]:
marine_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
marine_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille
print(marine_data)  # Afficher les données